In [1]:
#initialize packges
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install seaborn
!pip install matplotlib

import numpy as np
import pandas as panda 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

     |████████████████████████████████| 22.3 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 27.4 MB 91.9 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 90.9 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=0616ce5d643630809669a971b4b07024ceafb519bb5400376b52468bc103bf9b
  Stored in directory: /home/jovyan/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
     |████████████████████████████████| 285 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.3 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 40.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 79.4 MB/s eta 0:00:01


In [2]:
#add usefull functions
def convertfloatfromtitanic(value):    
        try:
            return float(value)
        except (ValueError, TypeError):
            return float(str(value).replace('.','',1))
        return default

In [3]:
database = panda.read_csv("titanicdb.csv")  #read database

In [4]:
database.info() #show databse information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id_passageiro             891 non-null    int64  
 1   sobrevivente              891 non-null    object 
 2   classe_da_passagem        891 non-null    int64  
 3   nome_passageiro           891 non-null    object 
 4   sexo                      891 non-null    object 
 5   idade                     714 non-null    float64
 6   irmaos_ou_esposa_a_bordo  891 non-null    int64  
 7   pais_filhos_a_bordo       891 non-null    int64  
 8   numero_passagem           891 non-null    object 
 9   valor_passagem            891 non-null    object 
 10  id_cabine                 204 non-null    object 
 11  local_de_embarque         889 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 83.7+ KB


In [5]:
total = database.isnull().sum().sort_values(ascending=False)
percent_1 = database.isnull().sum()/database.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = panda.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(12) #get null values percent in database ( missing data)

,Total,%
id_cabine,687,77.1
idade,177,19.9
local_de_embarque,2,0.2
id_passageiro,0,0.0
sobrevivente,0,0.0
classe_da_passagem,0,0.0
nome_passageiro,0,0.0
sexo,0,0.0
irmaos_ou_esposa_a_bordo,0,0.0
pais_filhos_a_bordo,0,0.0


In [6]:
 noinfodb = database.drop(columns=['id_passageiro', 'nome_passageiro','numero_passagem'])  #drop passenger id infos (eg. Name, id) 

In [7]:
noticket = noinfodb.drop(columns=['id_cabine'])  #drop cabin('id_cabine') column as have more missing data than real info 

In [8]:
noticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   sobrevivente              891 non-null    object 
 1   classe_da_passagem        891 non-null    int64  
 2   sexo                      891 non-null    object 
 3   idade                     714 non-null    float64
 4   irmaos_ou_esposa_a_bordo  891 non-null    int64  
 5   pais_filhos_a_bordo       891 non-null    int64  
 6   valor_passagem            891 non-null    object 
 7   local_de_embarque         889 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 55.8+ KB


In [9]:
gender = {"male" : 0, "female" : 1}
noticket["sexo"] = noticket["sexo"].map(gender) #convert object gender to int64 to make things more easy
    

In [10]:
noticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   sobrevivente              891 non-null    object 
 1   classe_da_passagem        891 non-null    int64  
 2   sexo                      891 non-null    int64  
 3   idade                     714 non-null    float64
 4   irmaos_ou_esposa_a_bordo  891 non-null    int64  
 5   pais_filhos_a_bordo       891 non-null    int64  
 6   valor_passagem            891 non-null    object 
 7   local_de_embarque         889 non-null    object 
dtypes: float64(1), int64(4), object(3)
memory usage: 55.8+ KB


In [11]:
noticket['Relatives']=noticket['pais_filhos_a_bordo'] + noticket['irmaos_ou_esposa_a_bordo']
noticket = noticket.drop(columns=['pais_filhos_a_bordo','irmaos_ou_esposa_a_bordo'])
#join relationships columns('pais_filhos_a_bordo' and 'irmaos_ou_esposa_a_bordo') into an single column Relatives 


In [12]:
noticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sobrevivente        891 non-null    object 
 1   classe_da_passagem  891 non-null    int64  
 2   sexo                891 non-null    int64  
 3   idade               714 non-null    float64
 4   valor_passagem      891 non-null    object 
 5   local_de_embarque   889 non-null    object 
 6   Relatives           891 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 48.9+ KB


In [13]:
noticket["local_de_embarque"].unique()
#get unique values from boarding 

array(['S', 'C', 'Q', nan], dtype=object)

In [14]:
boarding = {'S' : 0, 'C' : 1,'Q' : 2}
noticket['local_de_embarque'] = noticket['local_de_embarque'].fillna('U')
noticket['local_de_embarque'] = noticket['local_de_embarque'].map(boarding) 
#convert object boarding('local_de_embarque') to int64 to make things more easy to process


In [15]:
 noticket['valor_passagem'] = noticket['valor_passagem'].transform(func = lambda x : convertfloatfromtitanic(x)).astype(float)
 #convert object fare('valor_passagem') to float64 to make things more easy to process

In [16]:
noticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sobrevivente        891 non-null    object 
 1   classe_da_passagem  891 non-null    int64  
 2   sexo                891 non-null    int64  
 3   idade               714 non-null    float64
 4   valor_passagem      891 non-null    float64
 5   local_de_embarque   889 non-null    float64
 6   Relatives           891 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 48.9+ KB


In [22]:
X_train = noticket.drop("sobrevivente", axis=1)
Y_train = noticket["sobrevivente"]
X_test  = database.drop("id_passageiro", axis=1).copy()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   classe_da_passagem  891 non-null    int64  
 1   sexo                891 non-null    int64  
 2   idade               714 non-null    float64
 3   valor_passagem      891 non-null    float64
 4   local_de_embarque   889 non-null    float64
 5   Relatives           891 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 41.9 KB


AttributeError: 'Series' object has no attribute 'info'

In [20]:
sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)

sgd.score(X_train, Y_train)

acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [21]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').